# LLama2 chat example

In [1]:
from llama import ModelArgs, Tokenizer
from model_local import Transformer
from llama_local import Llama

from pathlib import Path

import torch
import time
import json

In [2]:
def load(ckpt_dir: str,
         tokenizer_path: str,
         max_seq_len: int,
         max_batch_size: int) -> Llama:
    
    print("Locating checkpoints.")
    checkpoints = sorted(Path(ckpt_dir).glob("*.pth"))

    assert(
        len(checkpoints) == 1
    ), f"Failed to load.  Only 7B model with 1 checkpoint is supported. But len(checkpoint) is {len(checkpoints)}."
    
    print("Loading checkpoint into memory... ", end="")
    _start_time = time.time()
    checkpoint = torch.load(checkpoints[0], map_location="cpu")
    print(f"done in {time.time() - _start_time:.2f} seconds.")
    
    print("Grabbing params...")
    with open(Path(ckpt_dir) / "params.json", "r") as f:
        params = json.loads(f.read())
        
    print("Loading model arguments...")
    model_args = ModelArgs(max_seq_len=max_seq_len, max_batch_size=max_batch_size, **params)
    
    
    print("Creating tokenizer...")
    tokenizer = Tokenizer(model_path=tokenizer_path)
    model_args.vocab_size = tokenizer.n_words
    print(f"model_args: {model_args}.")
    
    print("Creating transformer... ", end="")
    _start_time = time.time()
    model = Transformer(model_args)
    print(f"done in {time.time() - _start_time:.2f} seconds.")
    
    print("Loading checkpoint to model... ", end="")
    _start_time = time.time()
    model.load_state_dict(checkpoint, strict=False)
    print(f"done in {time.time() - _start_time:.2f} seconds.")
    
    print("Creating LLaMA generator...")
    generator = Llama(model, tokenizer)
    
        
    return generator

In [3]:
ckpt_dir = "../llama-2-7b-chat"
tokenizer_path = "../tokenizer.model"
max_seq_len = 1024
max_batch_size = 4

generator = load(ckpt_dir, tokenizer_path, max_seq_len, max_batch_size)


Locating checkpoints.
Loading checkpoint into memory... done in 4.95 seconds.
Grabbing params...
Loading model arguments...
Creating tokenizer...
model_args: ModelArgs(dim=4096, n_layers=32, n_heads=32, n_kv_heads=None, vocab_size=32000, multiple_of=256, ffn_dim_multiplier=None, norm_eps=1e-06, max_batch_size=4, max_seq_len=1024).
Creating transformer... done in 28.31 seconds.
Loading checkpoint to model... done in 2.88 seconds.
Creating LLaMA generator...


In [4]:
temperature: float = 0.6
top_p: float = 0.9

In [5]:
dialogs = [
        [{"role": "user", "content": "what is the recipe of mayonnaise?"}],
        [
            {"role": "user", "content": "I am going to Paris, what should I see?"},
            {
                "role": "assistant",
                "content": """\
Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
            },
            {"role": "user", "content": "What is so great about #1?"},
        ],
        [
            {"role": "system", "content": "Always answer with Haiku"},
            {"role": "user", "content": "I am going to Paris, what should I see?"},
        ],
        [
            {
                "role": "system",
                "content": "Always answer with emojis",
            },
            {"role": "user", "content": "How to go from Beijing to NY?"},
        ],
    ]

In [ ]:
results = generator.chat_completion(
        dialogs,  # type: ignore
        max_gen_len=None,
        temperature=temperature,
        top_p=top_p,
    )

In [ ]:
from llama.generation import sample_top_p

In [ ]:
sample_top_p